## Размерность (N, SxD, M)

In [203]:
import tensorflow as tf
import pandas as pd
import numpy as np

import keras
from keras import layers

In [204]:
dataset = pd.read_csv("dataset.csv").set_index("id")

In [205]:
dataset

,exam,lecture_1,lecture_2,lecture_3,practice_1,practice_2,practice_3,kt_1,kt_2
id,,,,,,,,,
0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0
0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0
0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0
0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0
0,0.4,1.000000,0.714286,0.75,0.500000,0.857143,0.75,0.0,0.0
...,...,...,...,...,...,...,...,...,...
61,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0
61,0.0,0.666667,0.000000,0.00,0.666667,0.000000,0.00,0.4,0.0
61,0.0,0.666667,0.000000,0.00,0.500000,0.000000,0.00,0.4,0.0


In [206]:
semesters_max_count = 8
disciplines_max_count = 8
features_count = 9

In [207]:
data = []
labels = []

In [208]:
zero_discipline_shape = [0.0 for i in range(features_count)]

In [209]:
for id in dataset.index:
    student = [[float(feature) for feature in list(discipline)] for discipline in dataset.loc[id].to_numpy()] 
    
    semesters_count = len(student)//semesters_max_count
    
    for semester_modif in range(semesters_count-1):
        last_semester = semesters_count-semester_modif
        aveatange_mark = sum([features[0] for features in student[(last_semester-1)*disciplines_max_count: last_semester*disciplines_max_count:]])/disciplines_max_count
        labels.append(aveatange_mark)
        
        data.append([[feature-0.5 for feature in discipline] for discipline in student[0:(last_semester-1)*disciplines_max_count:]] + 
                    [zero_discipline_shape for i in range(disciplines_max_count*(semesters_max_count-(last_semester-1)))])

In [210]:
labels = [round(i*5) for i in labels]

In [211]:
train_part = round(len(labels) * 0.85)
test_part = len(labels)-train_part

In [212]:
train_labels = keras.utils.to_categorical(labels[:train_part:])
test_labels = labels[train_part::]

train_data = np.array(data[:train_part:])
test_data = np.array(data[train_part::])

In [213]:
model = keras.Sequential()
model.add(layers.GRU(128, input_shape=(disciplines_max_count*semesters_max_count, features_count)))
model.add(layers.Dense(5, activation='softmax'))

/home/atom/Study/ГПО/ASU2304/training/first_model/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [214]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_data, train_labels, epochs=5, batch_size=16)

Epoch 1/5
887/887 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.5183 - loss: 1.1927
Epoch 2/5
887/887 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.7784 - loss: 0.5611
Epoch 3/5
887/887 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9057 - loss: 0.2746
Epoch 4/5
887/887 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9437 - loss: 0.1728
Epoch 5/5
887/887 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9583 - loss: 0.1323


In [215]:
test_model = model.predict(test_data)

79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


In [216]:
interpreate_test_model = []
for i in range(len(test_model)):
    ind_max = 0
    max = 0.0
    for j in range(len(test_model[i])):
        if max < float(test_model[i][j]):
            max = float(test_model[i][j])
            ind_max = j
    
    interpreate_test_model.append(ind_max)

In [217]:
test_compare = []
for i in range(len(test_model)):
    interpreate_test_model.append(ind_max)
    test_compare.append([interpreate_test_model[i], test_labels[i]])

In [218]:
test_compare

[[1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [3, 3],
 [1, 1],
 [0, 0],
 [4, 0],
 [0, 0],
 [4, 0],
 [0, 0],
 [4, 0],
 [0, 0],
 [4, 0],
 

In [219]:
right_predicions = 0
one_step_right_predictions = 0
two_step_right_predictions = 0

for i in test_compare:
    if i[0] == i[1]:
        right_predicions += 1
        one_step_right_predictions += 1
        two_step_right_predictions += 1

    if abs(i[0] - i[1]) == 1:
        one_step_right_predictions += 1
        two_step_right_predictions += 1

    if abs(i[0] - i[1]) == 2:
        two_step_right_predictions += 1

accuracy = right_predicions/len(test_labels)
one_step_accuracy = one_step_right_predictions/len(test_labels)
two_step_accuracy = two_step_right_predictions/len(test_labels)

In [220]:
print(accuracy)
print(one_step_accuracy)
print(two_step_accuracy)

0.6644027167399121
0.9168997203355973
0.9648421893727527


In [223]:
model.save("model.keras")